<a href="https://colab.research.google.com/github/mimingucci/ML/blob/main/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Init Network

We assume a sample neurol network with 2 layers include 1 hidden layer and 1 output layer

In [1]:
from random import seed
from random import random
def initialize_network(n_inputs, n_hiddens, n_outputs):
  network=list()
  hidden_layer=[{'weights':[random() for i in range(n_inputs+1)]} for j in range(n_hiddens)]
  network.append(hidden_layer)
  output_layer=[{'weights':[random() for i in range(n_hiddens+1)]} for j in range(n_outputs)]
  network.append(output_layer)
  return network
seed(1)
network=initialize_network(2, 1, 2)
for layer in network:
  print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


2.Activation

```
# This is formatted as code
```


Formula: activation=sum(weight*input)+bias

In [2]:
def activate(weights, inputs):
  activation=weights[-1]#bias
  for i in range(len(weights)-1):
    activation+=weights[i]*inputs[i]
  return activation

Transfer neurol using sigmoid, tanh, ... function

In [3]:
#using sigmoid function (z=1/(1+e^(-z)))
import math
def transfer(activation):
  return 1.0/(1.0+math.exp(-activation))

3. Forward Propagation

In [8]:
def forward_propagate(network, inputs):
  for layer in network:
    cur_network=list()
    for neurol in layer:
      activation=activate(neurol['weights'], inputs)
      neurol['output']=transfer(activation)
      cur_network.append(neurol['output'])
    inputs=cur_network
  return inputs

In [6]:
#test forward_propagate
print(network)
inputs=[1, 2]
outputs=forward_propagate(network, inputs)
print(outputs)

[[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}], [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
[0.6754093036624, 0.7445596419029368]


4. Backpropagation

```
# This is formatted as code
```
Call sigmoid function is f, we have:
```
```
Derivative of sigmoid function: (f)'=f*(1-f)

In [7]:
def derivative(f):
  return f*(1.0-f)

Error Backpropagation=(actual_output - expected)*derivative(actual_output) for output neurol

---

for hidden neurons error=(weight[k]* error[j])*deravative(output). Here, weight[k] represents weight of k'th neurol of following layer to current neuron, error[j] is the signal error from the j'th neuron in the output layer and output is output of current neuron.

In [9]:
# Backpropagate error and store in neurons
# We store error value in 'delta'
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * derivative(neuron['output'])

5. Train Network


```
# This is formatted as code
```
Here, this step is enable broke down into 2 sections:

1. Update Weights according to formula: weight=weight-learning_rate * error * input
2. Train using gradient descent




In [12]:
# Update network weights with error
def update_weights(network, row, lr):
	for i in range(len(network)):
		inputs = row[:-1]# because last element is label
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= lr * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= lr * neuron['delta'] # update bias

In [13]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) # squares errors
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error)) # trace

6. Predict

In [14]:
def predict(network, row):
  outputs=forward_propagate(network, row)
  return outputs.index(max(outputs))